# Week 6 Homework

***Due (pushed to your GitHub branch) on 10/25 by 11:59 pm***

## STAR two-pass alignment

Build the STAR SA:

In [8]:
%mkdir alignment/star_B/

!STAR \
--runThreadN 32 \
--runMode genomeGenerate \
--genomeDir ../2_genome_exploration/beegenome/star \
--genomeFastaFiles /data/users/waldhacw6865/BIOL343/2_genome_exploration/beegenome/genome.fa \
--sjdbGTFfile /data/users/waldhacw6865/BIOL343/2_genome_exploration/beegenome/genome_annotations.gtf \
--sjdbOverhang 169 \
--genomeSAindexNbases 12

mkdir: cannot create directory ‘alignment/star_B/’: File exists
	STAR --runThreadN 32 --runMode genomeGenerate --genomeDir ../2_genome_exploration/beegenome/star --genomeFastaFiles /data/users/waldhacw6865/BIOL343/2_genome_exploration/beegenome/genome.fa --sjdbGTFfile /data/users/waldhacw6865/BIOL343/2_genome_exploration/beegenome/genome_annotations.gtf --sjdbOverhang 169 --genomeSAindexNbases 12
	STAR version: 2.7.10b   compiled: 2022-11-01T09:53:26-04:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Oct 24 16:11:46 ..... started STAR run
Oct 24 16:11:46 ... starting to generate Genome files
Oct 24 16:11:49 ..... processing annotations GTF
Oct 24 16:11:53 ... starting to sort Suffix Array. This may take a long time...
Oct 24 16:11:54 ... sorting Suffix Array chunks and saving them to disk...
Oct 24 16:12:15 ... loading chunks from disk, packing SA...
Oct 24 16:12:21 ... finished generating suffix array
Oct 24 16:12:21 ... generating Suffix Array index
Oct 24 16:12:34 ... complete

Use STAR and ***all*** of your FASTQs for a first-pass alignment. Be sure to update the `sjdbOverhang` parameter to a value that is appropriate for your dataset. Rename the log file so that you can compare the logs from the first and second alignments:

In [13]:
!STAR \
--runThreadN 32 \
--runMode alignReads \
--genomeDir ../2_genome_exploration/beegenome/star \
--readFilesManifest manifestcopy_B.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outFileNamePrefix alignment/star_B/

	STAR --runThreadN 32 --runMode alignReads --genomeDir ../2_genome_exploration/beegenome/star --readFilesManifest manifestcopy_B.tsv --readFilesCommand zcat --outSAMtype BAM SortedByCoordinate --outSAMunmapped Within --outFileNamePrefix alignment/star_B/
	STAR version: 2.7.10b   compiled: 2022-11-01T09:53:26-04:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
Oct 24 16:23:15 ..... started STAR run
Oct 24 16:23:15 ..... loading genome
Oct 24 16:23:17 ..... started mapping
^C


Use STAR, all of your FASTQs, and `SJ.out.tab` for the second-pass alignment. You will likely need to make your own `manifest.tsv`:

In [ ]:
!STAR \
--runThreadN 32 \
--runMode alignReads \
--genomeDir ../2_genome_exploration/beegenome/star \
--readFilesManifest manifestcopy_B.tsv \
--readFilesCommand zcat \
--outSAMtype BAM SortedByCoordinate \
--outSAMunmapped Within \
--outSAMattributes NH HI AS nM RG \
--outFileNamePrefix alignment/star_B \
--sjdbFileChrStartEnd alignment/star/SJ.out.tab

!cp alignment/star/Log.final.out alignment/star/second-pass.final.out

Update this block to briefly describe the differences between the logs for the first and second alignments.

## HISAT alignment

(***Note***: you may need Dr. Wheeler to build the HISAT index of your genome. Ask him for assistance when you get to this part of the assignment.)

Use a Bash `while` loop to align all of your FASTQs and save the output files using the same names as the input FASTQ:

## Snakemake Pipeline

In contrast to last week (FASTQ files), there aren't as many decisions that need to be made during alignment, so using a notebook to control and report code isn't as necessary. As you saw in in [6_alignment](6_alignment.ipynb), file and directory management can get complicated during alignment. This is a case where Snakemake scripting shines.

1. Copy your `Snakefile` and `config.yaml` from `5_fastq` to `6_alignment`. 
2. In the `Snakefile`, add the following rules:  
    a. `fetch_genome` - using a variation from the code from `2_genome_exploration`, download your reference genome and annotations.
    b. `star_sa` - use `STAR` to produce the suffix array of the reference genome.  
    c. `star_align_first` - use `STAR` to perform first-pass alignment with all FASTQ files.
    d. `star_align_second` - use `STAR` to perform second-pass alignment with all FASTQ files.
    e. `hisat_align` - use `HISAT` to align all FASTQ files.
    e. `report` - use `multiqc` to aggregate all QC data into a single report.

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`.